In [3]:
import pandas as pd
import os
import re
from pathlib import Path
from typing import Dict, List, Optional, Any

In [4]:
# Fields to extract
ELEMENT_NAME_COL = 'Element Name'
VALUE_COL = 'Fact Value'
COMPANY_NAME_ELEMENT = 'NameOfTheCompany'
COMPLAINT_MECHANISM_ELEMENT = 'DescribeTheMechanismsInPlaceToReceiveAndRespondToConsumerComplaintsAndFeedbackExplanatoryTextBlock'

In [5]:
def get_text_value(df: pd.DataFrame, element_name: str) -> Optional[str]:
    if ELEMENT_NAME_COL not in df.columns or VALUE_COL not in df.columns:
        return None
    series = df.loc[df[ELEMENT_NAME_COL] == element_name, VALUE_COL]
    return str(series.iloc[0]) if not series.empty else None


def check_complaint_mechanisms_from_all_text(df: pd.DataFrame) -> Dict[str, int]:
    mechanism_flags = {mechanism: 0 for mechanism in MECHANISM_KEYWORDS.keys()}
    text_columns = df.select_dtypes(include='object')
    full_text = ' '.join(text_columns.fillna('').astype(str).values.flatten())

    for mechanism, pattern in PRECOMPILED_MECHANISM_REGEX.items():
        if pattern.search(full_text):
            mechanism_flags[mechanism] = 1
    return mechanism_flags


def extract_phone_numbers(text: Optional[str]) -> Optional[str]:
    if not text:
        return None
    numbers = PHONE_REGEX.findall(text)
    if numbers:
        cleaned_numbers = [num.strip() for num in numbers]
        unique_numbers = list(dict.fromkeys(cleaned_numbers))
        return ", ".join(unique_numbers)
    return None


def extract_consumer_complaint_info(directory: str, output_file: str) -> None:
    source_path = Path(directory)
    summary_data: List[Dict[str, Any]] = []

    if not source_path.is_dir() or not any(source_path.iterdir()):
        print(f"Error: The directory '{directory}' is empty or does not exist.")
        return

    print(f"[INFO] Starting analysis of files in '{directory}'...\n")

    for file_path in source_path.iterdir():
        if file_path.is_dir() or file_path.suffix not in ['.xlsx', '.xls', '.csv']:
            continue

        print(f"--- Processing File: {file_path.name} ---")

        try:
            if file_path.suffix in ['.xlsx', '.xls']:
                df = pd.read_excel(file_path, engine='openpyxl')
            else:
                df = pd.read_csv(file_path, on_bad_lines='skip', encoding_errors='ignore')

            company_name = get_text_value(df, COMPANY_NAME_ELEMENT) or "Company Name Not Found"
            complaint_text = get_text_value(df, COMPLAINT_MECHANISM_ELEMENT)

            mechanism_flags = check_complaint_mechanisms_from_all_text(df)
            extracted_phone = extract_phone_numbers(complaint_text)

            data_entry = {
                'Company Name': company_name,
                'Extracted Phone Number': extracted_phone,
                'Consumer Complaint Mechanism Text': complaint_text or "Mechanism Text Not Found"
            }

            data_entry.update(mechanism_flags)
            summary_data.append(data_entry)

            print(f"  Successfully extracted data for: {company_name}")
            if extracted_phone:
                print(f"  Extracted Phone: {extracted_phone}")

        except Exception as e:
            print(f"An error occurred while processing file {file_path.name}: {e}\n")

    if not summary_data:
        print("\nNo valid data could be processed from the files.")
        return

    summary_df = pd.DataFrame(summary_data)

    column_order = ['Company Name'] + list(MECHANISM_KEYWORDS.keys()) + ['Extracted Phone Number', 'Consumer Complaint Mechanism Text']
    summary_df = summary_df.reindex(columns=column_order)

    try:
        summary_df.to_excel(output_file, index=False, engine='openpyxl')

        print(f"\nSUCCESS: Master Excel sheet '{output_file}' has been created.")
        print(f"Total companies processed: {len(summary_df)}")

        print("\n--- SUMMARY STATISTICS (Top 5) ---")
        for mechanism in list(MECHANISM_KEYWORDS.keys())[:5]:
            count = summary_df[mechanism].sum()
            percentage = (count / len(summary_df)) * 100 if len(summary_df) > 0 else 0
            print(f"{mechanism}: {count} companies ({percentage:.1f}%)")
        print("...")

        print("\nFirst few rows of the consolidated data:")
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 120)
        pd.set_option('display.max_colwidth', 40)
        print(summary_df.head())

    except Exception as e:
        print(f"\nERROR: Could not save the Excel file '{output_file}'. Reason: {e}")


In [6]:
MECHANISM_KEYWORDS: Dict[str, List[str]] = {
    'In-Person / Branch Visit': ['in-person', 'branch visit', 'visit branch', 'physical visit', 'walk-in', 'office visit'],
    'Postal Mail': ['postal mail', 'post', 'letter'],
    'Written Correspondence': ['written correspondence', 'written complaint', 'grievance in writing', 'in writing'],
    'Social Media': ['social media', 'facebook', 'twitter', 'instagram', 'linkedin', 'whatsapp', 'telegram'],
    'Designated Grievance Officer / Nodal Team': ['grievance officer', 'nodal team', 'grievance cell', 'complaint officer', 'nodal officer'],
    'Website / Online Portal': ['website', 'online portal', 'web portal', 'online platform', 'internet', 'web-based'],
    'Phone / Toll-Free Helpline': ['phone', 'toll-free', 'helpline', 'telephone', 'call', 'hotline', 'customer care number'],
    'Email Communication': ['email', 'e-mail', 'electronic mail', 'email communication'],
    'Mobile App': ['mobile app', 'app store', 'play store', 'in-app', 'application'],
    'Chatbot / Live Chat': ['chatbot', 'live chat', 'web chat', 'chat bot', 'virtual assistant'],
    'Ombudsman / Regulatory Body': ['ombudsman', 'regulatory body', 'regulator', 'appellate authority'],
    'Fax': ['fax', 'facsimile'],
}

PRECOMPILED_MECHANISM_REGEX = {
    mechanism: re.compile('|'.join(r'\b' + re.escape(keyword) + r'\b' for keyword in keywords), re.IGNORECASE)
    for mechanism, keywords in MECHANISM_KEYWORDS.items()
}

PHONE_REGEX = re.compile(r"""
    (?:(?:\+91|0)?(?:[-\s]?)?)?      # Optional country code
    (?:(?:\(\d{2,5}\))?[-\s]?)?      # Optional STD code
    (?:(?:\d[-\s]?){7,12}\d)         # Digits with optional dashes/spaces
""", re.VERBOSE)

In [7]:

source_directory = 'excel_files'
output_excel_file = 'p9_complaints.xlsx'

if not os.path.exists(source_directory):
    os.makedirs(source_directory)
    print(f"Created directory: {source_directory}")
    print("Please add your company Excel/CSV files to this directory before running again.")

extract_consumer_complaint_info(source_directory, output_excel_file)

[INFO] Starting analysis of files in 'excel_files'...

--- Processing File: 360_ONE_WAM_LIMITED.xlsx ---
  Successfully extracted data for: 360 One Wam Limited
--- Processing File: 3I_Infotech_Limited.xlsx ---
  Successfully extracted data for: 3I Infotech Limited
--- Processing File: 3M_INDIA_LIMITED.xlsx ---
  Successfully extracted data for: 3M India Limited
--- Processing File: 5paisa_Capital_Limited.xlsx ---
  Successfully extracted data for: 5paisa Capital Limited
--- Processing File: 63_moons_technologies_limited.xlsx ---
  Successfully extracted data for: 63 Moons Technologies Limited
--- Processing File: Aarti_Drugs_Limited.xlsx ---
  Successfully extracted data for: Aarti Drugs Limited
--- Processing File: Aarti_Industries_Limited.xlsx ---
  Successfully extracted data for: Aarti Industries Limited
--- Processing File: Aarti_Pharmalabs_Limited.xlsx ---
  Successfully extracted data for: Aarti Pharmalabs Limited
--- Processing File: Aavas_Financiers_Limited.xlsx ---
  Successf